In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import numpy as np
from pyuvdata import UVCal, UVData
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, Angle
import pandas
import warnings 
import copy
from hera_notebook_templates import utils
import hera_qm
from hera_mc import cm_hookup
#warnings.filterwarnings('ignore')

In [ ]:
# get data location
JD = os.environ['JULIANDATE']
data_path = os.environ['DATA_PATH']
ant_metrics_ext = os.environ['ANT_METRICS_EXT']
print(f'JD = {JD}')
print(f'data_path = "{data_path}"')
print(f'ant_metrics_ext = "{ant_metrics_ext}"')

In [ ]:
# Load in data
HHfiles, difffiles, uvdx, uvdy = utils.load_data(data_path,JD)

## Autocorrelations for a single file

This plot shows autocorrelations for one timestamp of each antenna that is active and each polarization

In [ ]:
### plot autos
utils.plot_autos(uvdx, uvdy)

## Waterfalls of Autocorrelation Amplitudes for each Antenna and Each polarization

In [ ]:
uvd = UVData()
uvd.read(HHfiles, ant_str = 'autos', skip_bad_files=True)

In [ ]:
utils.plot_wfs(uvd, pol = 0)

In [ ]:
utils.plot_wfs(uvd, pol = 1)

## Correlation Metrics

The first plot shows the correlation metric (described below) for a set of baseline types, as calculated at several times throughout the night. It is expected that longer baselines (darker color) will exhibit lower values than the short baselines. 

The matrices show the phase correlation between antennas. Using the even and odd visibilities, each pixel is calculated as (even/abs(even)) * (conj(odd)/abs(odd)), and then averaged across time and frequency. If the phases are noise-like, this value will average down to zero. If the antennas are well correlated, the phases should not be noise-like, and this value should average to 1. The lines denoting node boundaries are intended to help confirm that inter-node correlations are functioning - if they aren't, this plot will appear block-diagonal.

This metric has shown to be LST locked - when comparing to other nights, be sure to compare for the same LST. It is expected that some LSTs will look much better or worse than others.

Note: Within each node, the order of antennas is determined by snap, and within that by snap input number. 

In [ ]:
badAnts = []
badAnts = utils.plotNodeAveragedSummary(uvd,HHfiles,JD)

Visibility amplitude spectra for a set of redundant baselines, labeled by inter vs. intranode baselines. The red and blue should exhibit the same bandpass shape - if the red are consistently different from the blue, this indicates an issue with internode correlations.

Note: antennas that were identified as bad by the correlation matrix have been removed from this plot.

In [ ]:
utils.plotVisibilitySpectra(HHfiles[len(HHfiles)//2], JD, badAnts=badAnts)

## Antenna Positions

Antennas outlined in black here have been identified by the correlation matrix as bad antennas. 

In [ ]:
uvd1 = UVData()
uvd1.read(HHfiles[5], skip_bad_files=True)
utils.plot_antenna_positions(uvd1, badAnts=badAnts)

## LST Coverage

In [ ]:
utils.plot_lst_coverage(uvd)

## Even and Odd File Checks

A waterfall showing the ratio between the even and odd visibilities. The purpose of this is to highlight xengine failures, which will cause this value to fall to zero or go to infinity. If things are working properly, this value should be stable at 1. The boundaries between different x-engines are shown by the vertical red lines.

In [ ]:
use_diffs = [f for f in difffiles if '%s/zen.%s.%s.sum.uvh5' % (data_path,f.split('.')[1],f.split('.')[2]) in HHfiles]
use_sums = [f for f in HHfiles if '%s/zen.%s.%s.diff.uvh5' % (data_path,f.split('.')[1],f.split('.')[2]) in use_diffs]
uvd_diff = UVData()
uvd_sum = UVData()
uvd_diff.read(use_diffs, ant_str = 'autos', skip_bad_files=True)
uvd_sum.read(use_sums, ant_str='autos',skip_bad_files=True)

utils.plotEvenOddWaterfalls(uvd_sum,uvd_diff)

## Single File Antenna Metrics

In [ ]:
antmetfiles = sorted(glob.glob(f'{data_path}/*{ant_metrics_ext}'))
ant_metrics = hera_qm.ant_metrics.load_antenna_metrics(antmetfiles[1])
utils.show_metric(ant_metrics, antmetfiles, ants=None, antpols=None, title='', ylabel='Modified z-Score', xlabel='')

## Antenna Metrics Over a Whole Night

In [ ]:
utils.all_ant_mets(antmetfiles, HHfiles)

## Delay spectrum

Delay spectrum CLEANed using uvtools.dspec.high_pass_fourier_filter with 7th-order Blackman-Harris window function and full frequency band. All delay spectra except for waterfalls are incoherently averaged over ~1h (if the observation is longer than 1h) with odd/even visibilities to remove noise bias.

In [ ]:
# bls = [(ant,ant) for ant in np.sort(uvd.get_ants())]
# _data_sq, data_rs, uvd_ds, uvd_diff = utils.clean_ds(HHfiles, difffiles, bls)

Diagnosis of delay spectra for three different delay regions:

Top panel: compute delay spectrum over 250-500 ns

Middle panel: compute standard deviation ratio $\sigma_{2500-3000}/\sigma_{3000-3200}$ to see the presence of spike at 2500-3000 ns. ~1 in y-axis (dashed line) means there is no spike, and >1 may indicate the presence of spike in the region

Bottom panel: compute distance between delay spectrum and noise level at large delay (3500-4000 ns). ~0 (dashed line) means the delay spectrum can go down to the noise level

In [ ]:
# utils.plot_ds_diagnosis(uvd_diff, _data_sq)

In [ ]:
# utils.plot_ds_nodes(uvd_diff, _data_sq)

In [ ]:
# utils.plot_wfds(uvd_ds, _data_sq, 'ee')

In [ ]:
# utils.plot_wfds(uvd_ds, _data_sq, 'nn')

These plots show incoherently averaged (~1h) delay spectra, autocorrelations and clean residuals for each antenna.

Left panel: delay spectra of autocorrelations, averaged noises from diff files (|$<\tilde{\Delta}>_{\rm t}$|, thiner fluctuating lines). The variance of the delay spectrum is also shown ($\sqrt{<|\tilde{\Delta}|^2>_{\rm t}/(2N_{\rm t})}$) which is consistent with the average of diff. Three shaded zones are those explained in the above diagnosis plots.
    
Top right: autocorrelations w/o and w/ flagging. Autocorrelations are normalized to have median=1. Flagged ones are shifted for clarity.
    
Bottom right: clean residuals normalized by the autocorrelations. If there are unflagged RFIs, which may affect the delay spectrum, we may see remaining RFI more clearly in the plot.

In [ ]:
# utils.plot_ds(uvd_ds, uvd_diff, _data_sq, data_rs)

In [ ]:
# bls_cr = [(uvd.get_ants()[0],ant) for ant in np.sort(uvd.get_ants())][1:]
# _data_sq_cr, data_rs_cr, uvd_cr, uvd_diff_cr = utils.clean_ds(HHfiles, difffiles, bls_cr)

Waterfalls for delay spectrum of crosscorrelation with a reference antenna

In [ ]:
# utils.plot_wfds_cr(uvd_cr, _data_sq_cr, 'ee')

In [ ]:
# utils.plot_wfds_cr(uvd_cr, _data_sq_cr, 'nn')